In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Liste pour stocker toutes les données
toutes_donnees = []

"""Scrape toutes les pages jusqu'à ce qu'il n'y ait plus de contenu"""
page = 1
has_next_page = True

while has_next_page:

    print(f"Traitement de la page {page} du site Tayara ...")

    # URL de la page courante
    url_page = f"https://www.tayara.tn/ads/c/Véhicules/Voitures/?page={page}"

    # Requête pour la page liste
    response = requests.get(url_page)
    if response.status_code != 200:
      print(f"Erreur HTTP page {page}")
      continue
    soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver toutes les annonces
    annonces = soup.find_all('article')

    if not annonces:
        print("Plus d'annonces trouvées, fin du scraping.")
        has_next_page = False
        break

    print(f" {len(annonces)} annonces de voitures trouvées dans les site Tayara")

    # Traiter chaque annonce
    for i, annonce in enumerate(annonces):
        try:
            # Récupérer le lien vers la page détaillée
            lien = annonce.find('a')['href']
            # annonce.find('a') : Cherche la balise <a> dans l'annonce
            # ['href'] : Prend la valeur de l'attribut href
            # => Le chemin vers la page de détails de la voiture de l'annonce

            url_detail = "https://www.tayara.tn" + lien #concaténation pour accéder à la voiture sélectionnée

            # Aller sur la page détaillée
            response_detail = requests.get(url_detail)
            soup_detail = BeautifulSoup(response_detail.content, 'html.parser')
            #html.parser comprend le code HTML et le transforme en arbre compréhensible par python

            # Extraire le titre et prix
            titre = annonce.find('h2')
            prix = annonce.find('data') #le prix se trouve dans balise data

            # Dictionnaire pour cette annonce avec les informations de base
            donnees_annonce = {
                'Titre': titre.text.strip() if titre else 'N/A',
                'Prix': prix.text.strip() if prix else 'N/A',
                'Page': page,
                'URL': url_detail
            }

            # EXTRAIRE LES 11 CARACTÉRISTIQUES DE LA VOITURE
            features = soup_detail.find_all('li', class_='col-span-6') #features de voiture sous forme de li avec col-span-6 - on peut trouver 11 au max (10 possible ....)

            for feature in features:
                label = feature.find('span', class_='text-gray-600/80') #label: titre caractéristique
                valeur = feature.find('span', class_='text-gray-700/80') #valeur: valeur caractéristique

                if label and valeur:
                    donnees_annonce[label.text.strip()] = valeur.text.strip() #supprimer les espaces inutiles

            # Ajouter aux données
            toutes_donnees.append(donnees_annonce)
            print(f" Annonce {i+1} a {len(features)} caractéristiques")



        except Exception as e:
            print(f"Erreur annonce {i+1}: {e}")
            continue

    print(f"Page {page} terminée\n")
    # Pause pour éviter de surcharger le site
    time.sleep(1)
    page += 1  # Passes à la page suivante


# Sauvegarder toutes les données en CSV
if toutes_donnees:
    with open('tayara_scrapping.csv', 'w', newline='', encoding='utf-8') as f:
        # Obtenir toutes les colonnes
        colonnes = set()


        # Parcourir CHAQUE annonce pour trouver TOUTES les colonnes possibles
        for annonce in toutes_donnees:
            colonnes.update(annonce.keys())

        print("Colonnes trouvées:", colonnes)
        #pour chaque annonce de voiture on peut trouver diff colonnes selon le nombre de caractéristiques qu'elle a - les colonnes 'Page', 'Titre', 'Prix', 'URL' sont toujours présents

        # Ordonner : d'abord les colonnes fixes, puis les caractéristiques
        colonnes_fixes = ['Page', 'Titre', 'Prix', 'URL']
        colonnes_caracteristiques = [c for c in colonnes if c not in colonnes_fixes]
        colonnes_ordonnees = colonnes_fixes + colonnes_caracteristiques

        #Crée un objet qui sait insérer des dictionnaires dans un fichier CSV, avec un ordre de colonnes
        writer = csv.DictWriter(f, fieldnames=colonnes_ordonnees)
        #Écris la première ligne du CSV avec les noms des colonnes
        writer.writeheader()
        #Écris toutes les lignes de données d'un coup
        writer.writerows(toutes_donnees)

    print(f"{len(toutes_donnees)} annonces sauvegardées")
else:
    print("Aucune donnée trouvée")